## Item 26: Use Multiple Inheritance Only for Mix-in Utility Classes

In [ ]:
import json

from pprint import pprint

* Python is an object-oriented language with built-in facilities for making multiple inheritance tractable.
    * See `Item 25`: Initialize Parent Classes with `super`.
* However, it's better to avoid multiple inheritance altogether.
* Instead multiple inheritance, consider writing a `mix-in` instead.

### Mixins (multiple inheritance)

* Inherited from multiple classes.
* A `mix-in` is a small class that only defines a set of additional methods that a class should provide.
* `Mix-in` classes don't define their own instance attributes nor require their `__init__` constructor to be called.

* Uses
    * enable functionality for framework such as Django.
    * streamline repetitious operatioms.

* `Min-ins` can be composed and layered to minimize repetitive code and maximize reuse.

* eg
    * Convert a Python object from its in-memory representation to a dictionary that's ready for serialization.
    * Write a generic function so you can use it with all of your classes.
    * `isinstance` is going to tell did you inherited from this class.

In [ ]:
class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

* Defind an example class that uses the `mix-in` to make a dictionary representation of a binary tree.

In [ ]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

* Translating a large number of related Python objects into a dictionary becoes easy.

In [ ]:
tree = BinaryTree(10,
    left=BinaryTree(7, right=BinaryTree(9)),
    right=BinaryTree(13, left=BinaryTree(11))
)

In [ ]:
# dir(tree)

In [ ]:
tree.left

In [ ]:
tree.right

In [ ]:
tree.value

In [ ]:
tree.to_dict()

* The best part about `mix-ins` is that you can make their generic functionality pluggable so behaviors can be overridden when required.

* Define a subclass of `BinaryTree` that holds a reference to its parent.
* This circular reference would cause the default implementation of `ToMixin.to_dict` loop forever.

* Solution
    * Override the `ToDictMixin._traverse` method in the `BinaryTreeWithParent` class to only process values that matter.
    * Override the `_traverse` method to not traverse the parent and just insert its numerical value.

In [ ]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, 
                 left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
    
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and 
               key == "parent"):
            return value.value  # prevent cycles
        else:
            return super()._traverse(key, value)

* Calling `BinaryTreeWithParent.to_dict` will work without issue because the circular referencing properties aren't followed.

In [ ]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)

In [ ]:
print(root.to_dict())

In [ ]:
pprint(root.to_dict())

* By defining `BinaryTreeWithParent._traverse`, i've enabled any class that has an attribute of type `BinaryTreeWithParent` to automatically work with `ToDictMixin`.

In [ ]:
class NamedSubTree(ToDictMixin):
    def __init__(self, name, tree_with_parent):
        self.name = name
        self.tree_with_parent = tree_with_parent

In [ ]:
my_tree = NamedSubTree("foobar", root.left.right)

In [ ]:
pprint(my_tree.to_dict())  # no infinite loop

### JSON serialization

* Create mix-ins that provides generic JSON serialization for any class.
* You can do this by assuing that a class provides a `to_dict` method (may or may not be provided by the `ToDictMixin` class).
* The `JsonMixin` class defines both instance methods and class methods. 
* In this example, only requirements are that the class has a `to_dict` method and its `__init__` method takes keyword arguments.
     * See `Item 19`: Provide Optional Behavior with Keyword Arguments.

In [ ]:
class JsonMixin:
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())

* This `Mix-in` makes it simple to create hierarchies of utility classes that can be serialized to and from JSON with little boilerplate.

In [ ]:
class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [
            Machine(**kwargs) for kwargs in machines]
        
    
class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports=None, speed=None):
        self.ports = ports
        self.speed = speed
        
        
class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk

* Serializing these classes to and from JSON is simple.
* Verify that the data is able to send round-trip through serializing and deserializing.

In [ ]:
serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12},
        {"cores": 2, "ram": 4e9, "disk": 500e9}
    ]
}"""
serialized

In [ ]:
deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()

In [ ]:
json.loads(serialized)

In [ ]:
json.loads(roundtrip)

In [ ]:
assert json.loads(serialized) == json.loads(roundtrip)

### Things to Remember

* Avoid using multiple inheritance if `mix-in` classes can achieve the same outcome.
* Use pluggable behaviors at the instance level to provide per-class customization when `mix-in` classes may require it.
* Compose `mix-ins` to create complex functionality from simple behaviors.